# Technical Breakdown of Predictive Models

### Cluster Analysis of Flavors

In [1]:
import pandas as pd
import re
import numpy as np
import os
%matplotlib inline
import matplotlib.pyplot as plt
import string
from flavors import *

In [2]:
lower_flavor = []
for w in flavor_list:
    #print(w.lower())
    lower_flavor.append(w.lower())
#print(lower_flavor)
flavor_list = lower_flavor
print(flavor_list)

['flower', 'iris', 'peony', 'elderflower', 'acacia', 'lilac', 'jasmine', 'honeysuckle', 'violet', 'lavender', 'rose', 'potpourri', 'hibiscus', 'citrus', 'lime', 'lemon', 'grapefruit', 'orange', 'marmalade', 'tree fruit', 'quince', 'apple', 'pear', 'nectarine', 'peach', 'apricot', 'persimmon', 'tropical fruit', 'pineapple', 'mango', 'guava', 'passion fruit', 'lychee', 'bubblegum', 'red fruit', 'cranberry', 'red plum', 'pomegranate', 'sour cherry', 'strawberry', 'cherry', 'raspberry', 'black fruit', 'boysenberry', 'black currant', 'black cherry', 'plum', 'blackberry', 'blueberry', 'olive', 'dried fruit', 'raisin', 'fig', 'date', 'fruitcake', 'noble rot', 'beeswax', 'ginger', 'honey', 'spice', 'white pepper', 'red pepper', 'black pepper', 'cinnamon', 'anise', '5-spice', 'fennel', 'eucalyptus', 'mint', 'thyme', 'vegetable', 'grass', 'tomato leaf', 'gooseberry', 'bell pepper', 'jalapeño', 'bitter almond', 'tomato', 'sun-dried tomato', 'black tea', 'earth', 'clay pot', 'slate', 'wet gravel',

In [3]:
df = pd.read_csv(os.path.join("wineData.csv"))
df.head()

,obs_id,title,winery,country,province,variety,description,points,price
0,0,Nicosia 2013 Vulkà Bianco (Etna),Nicosia,Italy,Sicily & Sardinia,White Blend,"Aromas include tropical fruit, broom, brimston...",87,NaN
1,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos,Portugal,Douro,Portuguese Red,"This is ripe and fruity, a wine that is smooth...",87,15.0
2,2,Rainstorm 2013 Pinot Gris (Willamette Valley),Rainstorm,US,Oregon,Pinot Gris,"Tart and snappy, the flavors of lime flesh and...",87,14.0
3,3,St. Julian 2013 Reserve Late Harvest Riesling ...,St. Julian,US,Michigan,Riesling,"Pineapple rind, lemon pith and orange blossom ...",87,13.0
4,4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Sweet Cheeks,US,Oregon,Pinot Noir,"Much like the regular bottling from 2012, this...",87,65.0


In [4]:
desc_df = df[["description", "points"]]
desc_df.head()

,description,points
0,"Aromas include tropical fruit, broom, brimston...",87
1,"This is ripe and fruity, a wine that is smooth...",87
2,"Tart and snappy, the flavors of lime flesh and...",87
3,"Pineapple rind, lemon pith and orange blossom ...",87
4,"Much like the regular bottling from 2012, this...",87


In [5]:
desc_df.count()

description    129971
points         129971
dtype: int64

In [6]:
desc_df.head()

,description,points
0,"Aromas include tropical fruit, broom, brimston...",87
1,"This is ripe and fruity, a wine that is smooth...",87
2,"Tart and snappy, the flavors of lime flesh and...",87
3,"Pineapple rind, lemon pith and orange blossom ...",87
4,"Much like the regular bottling from 2012, this...",87


In [7]:
def word_cleaner(word):
    return word in flavor_list

In [8]:
def func_to_apply_to_each_row(cell):
    line = re.sub(r'[^\w\s]','',cell)
    #return list(filter(word_cleaner, line.split(" ")))
    lst = list(filter(word_cleaner, line.split(" ")))
    lst = list(set(lst))
    return (' '.join(lst))

In [9]:
#apply applies a function to every row, with the specified column
desc_df['clean_list'] = desc_df['description'].apply(func_to_apply_to_each_row)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
desc_df.head()

,description,points,clean_list
0,"Aromas include tropical fruit, broom, brimston...",87,citrus apple
1,"This is ripe and fruity, a wine that is smooth...",87,
2,"Tart and snappy, the flavors of lime flesh and...",87,lime pineapple
3,"Pineapple rind, lemon pith and orange blossom ...",87,guava mango orange lemon
4,"Much like the regular bottling from 2012, this...",87,


In [ ]:
# csvpath = "flavor_data.csv"
# #desc_df.to_csv(csvpath)

In [11]:
flavorData = list(desc_df.clean_list.values)

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer , CountVectorizer

In [13]:
tfidf_vectorizer = TfidfVectorizer(min_df=2, max_df=0.95, max_features=1000)
tfidf = tfidf_vectorizer.fit_transform(flavorData)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

In [14]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000)
tf = tf_vectorizer.fit_transform(flavorData)
tf_feature_names = tf_vectorizer.get_feature_names()

In [15]:
from sklearn.decomposition import NMF , LatentDirichletAllocation
nmf = NMF(n_components = 28 , random_state=1 , alpha=.1 , l1_ratio=.5 , init='nndsvd').fit(tfidf)
lda = LatentDirichletAllocation(n_components=28 , max_iter=10 , learning_method='online' , learning_offset=50., random_state=0)
lda_v = lda.fit(tf)

In [16]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

no_top_words = 10
display_topics(nmf, tfidf_feature_names, no_top_words)
display_topics(lda_v, tf_feature_names, no_top_words)

Topic 0:
cherry violet anise rose pomegranate cocoa mushroom truffle eucalyptus thyme
Topic 1:
spice violet cocoa rose lychee raisin ginger anise tomato boysenberry
Topic 2:
apple nectarine butter ginger honeysuckle cream acacia quince jasmine beeswax
Topic 3:
blackberry raisin boysenberry cocoa anise olive coconut fig violet lavender
Topic 4:
plum tomato raisin olive cocoa fig boysenberry violet coconut thyme
Topic 5:
citrus nectarine grass gooseberry lychee fig tomato honeysuckle beeswax acacia
Topic 6:
vanilla cream coconut butter cocoa honeysuckle dill raisin fig boysenberry
Topic 7:
raspberry rose violet pomegranate cocoa tomato anise boysenberry iris thyme
Topic 8:
lemon butter honeysuckle cream grass nectarine quince ginger beeswax jasmine
Topic 9:
peach nectarine honeysuckle mango jasmine rose butter lychee tomato grass
Topic 10:
pear nectarine butter honeysuckle quince cream ginger jasmine fig beeswax
Topic 11:
tobacco olive raisin anise violet fig tomato cocoa coconut truffle

In [17]:
import pyLDAvis.sklearn

In [18]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer, mds='tsne')

In [19]:
panel

PreparedData(topic_coordinates=           Freq  cluster  topics           x           y
topic                                                   
17     8.063109        1       1   -0.877660  -31.491488
24     6.588447        1       2    9.599596   49.898350
12     6.503964        1       3  -49.286434   61.249660
2      6.479694        1       4  -62.234554 -126.931442
0      4.882461        1       5   70.373665 -118.353027
9      4.671930        1       6  -13.868026   99.534966
19     4.402661        1       7   91.995644   -1.662310
13     4.265204        1       8  -33.412617  -75.605957
3      4.081828        1       9    4.627056  155.276581
25     3.957303        1      10  -61.238113  -29.249212
16     3.658827        1      11 -100.528267  -76.145950
11     3.361715        1      12   69.866592   47.706161
15     3.329759        1      13    3.699733 -136.767288
6      3.074909        1      14  -25.740103   12.574601
14     2.979779        1      15 -139.790161   41.033176
27     2.972299        1      16   58.050980  -42.825573
1      2.778709        1      17  -83.306007   20.117088
23     2.757915        1      18   42.093483   94.099060
22     2.684678        1      19   34.463875    5.821041
7      2.453878        1      20  136.918610   41.579151
4      2.452131        1      21   70.564674  145.677200
21     2.165224        1      22  -61.861057  136.483475
26     2.119377        1      23  108.988899   94.841705
10     2.042224        1      24 -128.444672  -22.942677
20     1.910888        1      25   22.637163  -81.105766
18     1.892882        1      26 -100.729515   89.222527
5      1.875521        1      27  109.255333  -70.993965
8      1.592684        1      28  148.337692  -22.607941, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
72    Default  14783.000000       spice  14783.000000  30.0000  30.0000
11    Default  21116.000000      cherry  21116.000000  29.0000  29.0000
13    Default   9455.000000      citrus   9455.000000  28.0000  28.0000
79    Default  10155.000000     vanilla  10155.000000  27.0000  27.0000
2     Default  10421.000000       apple  10421.000000  26.0000  26.0000
5     Default  10056.000000  blackberry  10056.000000  25.0000  25.0000
62    Default  10456.000000        plum  10456.000000  24.0000  24.0000
57    Default   6877.000000        pear   6877.000000  23.0000  23.0000
56    Default   7761.000000       peach   7761.000000  22.0000  22.0000
44    Default   7414.000000       lemon   7414.000000  21.0000  21.0000
67    Default   7319.000000   raspberry   7319.000000  20.0000  20.0000
55    Default   5534.000000      orange   5534.000000  19.0000  19.0000
37    Default   4999.000000       honey   4999.000000  18.0000  18.0000
46    Default   5024.000000        lime   5024.000000  17.0000  17.0000
43    Default   5306.000000     leather   5306.000000  16.0000  16.0000
70    Default   4329.000000       smoke   4329.000000  15.0000  15.0000
3     Default   4482.000000     apricot   4482.000000  14.0000  14.0000
12    Default   4317.000000    cinnamon   4317.000000  13.0000  13.0000
16    Default   4465.000000      coffee   4465.000000  12.0000  12.0000
75    Default   5106.000000     tobacco   5106.000000  11.0000  11.0000
33    Default   4123.000000  grapefruit   4123.000000  10.0000  10.0000
73    Default   4045.000000  strawberry   4045.000000   9.0000   9.0000
23    Default   3743.000000       earth   3743.000000   8.0000   8.0000
61    Default   3646.000000   pineapple   3646.000000   7.0000   7.0000
6     Default   3317.000000   blueberry   3317.000000   6.0000   6.0000
1     Default   3052.000000       anise   3052.000000   5.0000   5.0000
19    Default   2788.000000   cranberry   2788.000000   4.0000   4.0000
28    Default   2778.000000      flower   2778.000000   3.0000   3.0000
66    Default   2083.000000      raisin   2083.000000   2.0000   2.0000
53    Default  

### Linear Model Using the Continuous Variables from the Data

In [16]:
from sklearn import linear_model
from sklearn.metrics import r2_score
reg = linear_model.Ridge()
model = reg.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f'R-Squared for test data: {model.score(X_test, y_test)}')
print(f'R-Squared for predictions: {r2_score(y_test, y_pred)}')

R-Squared for test data: 0.13880089747170443
R-Squared for predictions: 0.13880089747170443


* Predicted scores are only weakly correlated to test data

### Classifier Models

### First Attempt - A Ridge Classifier Model

In [15]:
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import r2_score
clf = RidgeClassifier().fit(X_train, y_train)
print(f'R-Squared for test data: {clf.score(X_test, y_test)}')
y_pred = clf.predict(X_test)
print(f'R-Squared for predictions: {r2_score(y_test, y_pred)}')

R-Squared for test data: 0.620746319753509
R-Squared for predictions: 0.12766027469626617


### Second Attempt - A Decision Tree Model

In [16]:
from sklearn import tree
dt = tree.DecisionTreeClassifier()
dt = clf.fit(X_train, y_train)
print(f'R-Squared for test data: {dt.score(X_test, y_test)}')
y_pred = dt.predict(X_test)
print(f'R-Squared for predictions: {r2_score(y_test, y_pred)}')

R-Squared for test data: 0.620746319753509
R-Squared for predictions: 0.12766027469626617


### Third Attempt - A Random Forest Model

In [17]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
print(f'R-Squared for test data: {rf.score(X_test, y_test)}')
y_pred = rf.predict(X_test)
print(f'R-Squared for predictions: {r2_score(y_test, y_pred)}')

R-Squared for test data: 0.6457377610407394
R-Squared for predictions: 0.13993908775622077


In [18]:
# List of IV features by importance
sorted(zip(rf.feature_importances_, X_test), reverse=True)

[(0.5044792019960175, 'price'),
 (0.37958675850207435, 'word_count'),
 (0.11593403950190773, 'age')]

### Final Attempt - Random Forest With Strongest Predictor (Price)

In [19]:
X2 = df['price'].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X2, encoded_y, random_state=42)
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
print(f'R-Squared for test data: {rf.score(X_test, y_test)}')
y_pred = rf.predict(X_test)
print(f'R-Squared for predictions: {r2_score(y_test, y_pred)}')

R-Squared for test data: 0.6337897980143786
R-Squared for predictions: 0.12276429227362606


### Limitations
* Too many values in our categorical Variables
* Subjectivity of wine scoring practices
* Reviews written by amateur reviewers

### Ultimately machine learning will not cost any sommeliers their job
* Wine quality assessments appear to be very subjective
* There is a lot of variability in the relationships between score and other descriptive variables
* After trying numerous types of models and data manipulation practices, none of our models were able to assess wine quality

In [20]:
%%HTML
<html><head><script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.3.4/require.min.js" integrity="sha256-Ae2Vz/4ePdIu6ZyI/5ZGsYnb+m0JlOmKPjt6XZ9JJkA=" crossorigin="anonymous"></script><script src="https://unpkg.com/@jupyter-widgets/html-manager@*/dist/embed-amd.js" crossorigin="anonymous"></script><script type="application/vnd.jupyter.widget-state+json">{"version_major": 2, "version_minor": 0, "state":{}}</script></head><body><iframe src="https://giphy.com/embed/E3L5goMMSoAAo" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/amy-schumer-E3L5goMMSoAAo">via GIPHY</a></p></body></html>